# From OMOP to IDEA4RC Surgery

In [ ]:
# Conexión a la base de datos OMOP CDM
conn = psycopg2.connect(
    dbname="omopdb",
    user="postgres",
    password="mysecretpassword",
    host="localhost",
    port="5432"
)

# Comando para hacer el mapeo (insercción de datos del IDEA4RC.csv a OMOP)
curOMOP = conn.cursor()
config = {
    'user': 'user', 
    'password': 'password',
    'host': '127.0.0.1',
    'database': 'idea4rc_dm',
    'raise_on_warnings': True
}

conn2 = mysql.connector.connect(**config)
curIDEA = conn2.cursor()

### From ProcedureOccurrence to IDEA

I do not know how to get the H&N stage since we do not have any concept id to help us and it is not clear which one in Athena would be the correct one. Same with Metastasis

TO BE DONE

In [ ]:
selectEpisodes = "SELECT DISTINCT episode_id, episode_start_date FROM episode_event"
curOMOP.execute(selectEpisodes)
episodes = curOMOP.fetchall()
selectProcedureOccurrence = """SELECT procedure_date, procedure_concept_id FROM procedure_occurrence INNER JOIN episode_event ON procedure_occurrence.visit_occurrence_id = episode_event.event_id
WHERE episode_id = %s AND procedure_concept_id IN (4301351,4315400,4217482,4181193)"""
episodeIDEA="""
    SELECT id FROM cancer_episode WHERE patient = %s AND start_date = %s"""
insert= """INSERT INTO surgery (episode_event, procedure_concept_id, visit_occurrence_id) VALUES (%s, %s, %s)"""
for episode in episodes:
    curIDEA.execute(episodeIDEA, (episode[0], episode[1]))
    episode_id = curIDEA.fetchone()
    curOMOP.execute(selectProcedureOccurrence, (episode_id[0]))
    procedures = curOMOP.fetchall()

### From Measurement to IDEA
Here we find a similar problem, since we need to find those 